# Imports
All problem-speciffic files (code) in in the folder `HyperElasticity3D`, namely the loading of pre-prepared mesh and definition of energy in jax.numpy.

In [1]:
from tools.minimizers import newton
from tools.sparse_solvers import HessSolverGenerator
from tools.jax_diff import EnergyDerivator as EnergyDerivator
from HyperElasticity3D.jax_energy import J
from HyperElasticity3D.mesh import MeshHyperElasticity3D
from HyperElasticity3D.rotate_boundary import rotate_boundary

import numpy as np
from jax import config

config.update("jax_enable_x64", True)

# Setup of the problem

In [2]:
rotation_per_iter = 4 * 2 * np.pi / 24
mesh = MeshHyperElasticity3D(mesh_level=3)
params, adjacency, u_init = mesh.get_data_jax()
energy_hyperelasticity = EnergyDerivator(J, params, adjacency, u_init)

params = rotate_boundary(params, angle=rotation_per_iter)
energy_hyperelasticity.params = params

F, dF, ddF = energy_hyperelasticity.get_derivatives()
ddf_with_solver = HessSolverGenerator(ddf=ddF, solver_type="amg", elastic_kernel=mesh.elastic_kernel, verbose=True, tol=1e-3)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


# Solution via Newton's method

In [3]:
res = newton(F, dF, ddf_with_solver, u_init, verbose=True, tolf=1e-4, linesearch_tol=1e-3)
res

Iterations in AMG solver: 6.
it=1, f=34.45339, fstep = 2.15209e+01, ||g||=3090.47754, nitf=17, a=3.28240e-01
Iterations in AMG solver: 5.
it=2, f=5.71812, fstep = 2.87353e+01, ||g||=2474.30769, nitf=17, a=9.66394e-01
Iterations in AMG solver: 8.
it=3, f=2.12818, fstep = 3.58995e+00, ||g||=915.46300, nitf=17, a=1.02694e+00
Iterations in AMG solver: 13.
it=4, f=0.98581, fstep = 1.14237e+00, ||g||=444.02223, nitf=17, a=1.21738e+00
Iterations in AMG solver: 12.
it=5, f=0.72635, fstep = 2.59461e-01, ||g||=177.05774, nitf=17, a=2.43103e-01
Iterations in AMG solver: 9.
it=6, f=0.45953, fstep = 2.66822e-01, ||g||=187.33275, nitf=17, a=1.41611e+00
Iterations in AMG solver: 26.
it=7, f=0.45712, fstep = 2.40404e-03, ||g||=97.07774, nitf=17, a=-3.87707e-02
Iterations in AMG solver: 19.
it=8, f=0.39618, fstep = 6.09390e-02, ||g||=98.39233, nitf=17, a=4.10946e-01
Iterations in AMG solver: 15.
it=9, f=0.32681, fstep = 6.93753e-02, ||g||=86.90240, nitf=17, a=1.02371e+00
Iterations in AMG solver: 13.
i

{'x': Array([ 0.00124997, -0.00501302, -0.00498591, ...,  0.39874986,
         0.00683569, -0.00181169], dtype=float64),
 'fun': array(0.16255661),
 'nit': 23,
 'time': 63.855648853990715,
 'message': 'Stopping condition for f is satisfied'}

### And other stepps of rotation will be as follows:

In [ ]:
# Apply timestep = rotation of the right boundary
params = rotate_boundary(params, angle=rotation_per_iter)
energy_hyperelasticity.params = params

# Get derivatives
F, dF, ddF = energy_hyperelasticity.get_derivatives()
ddf_with_solver = HessSolverGenerator(ddf=ddF, solver_type="amg", elastic_kernel=mesh.elastic_kernel, verbose=True, tol=1e-3)
# Actualize initial guess
u_init = res["x"]

# Newton method
res = newton(F, dF, ddf_with_solver, u_init, verbose=True, tolf=1e-4, linesearch_tol=1e-3)
res